# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [3]:
os.environ["SPECTRAL_CONNECTIVITY_ENABLE_GPU"] = "true"
import cupy as cp

In [4]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [5]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [6]:
from sklearn.metrics import confusion_matrix

In [7]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [8]:
FONTSIZE = 20

In [9]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

# Functions

In [10]:
def generate_pairs(lst):
    """
    Generates all unique pairs from a list.

    Parameters:
    - lst (list): The list to generate pairs from.

    Returns:
    - list: A list of tuples, each containing a unique pair from the input list.
    """
    n = len(lst)
    return [(lst[i], lst[j]) for i in range(n) for j in range(i+1, n)]

## Inputs & Data

Explanation of each input and where it comes from.

In [11]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

In [12]:
TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 1
TIME_WINDOW_STEP = 0.5
RESAMPLE_RATE=1000

In [13]:
LFP_TRACES_DF = pd.read_pickle("/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/rce_pilot_2_01_lfp_traces_and_frames.pkl")

In [14]:
LFP_TRACES_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,...,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace
0,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.933348, 0.8057418, 0.9461086, 1.2687699, 1....","[0.44765243, 0.479771, 0.7427417, 0.97560126, ...","[0.89443207, 0.96188104, 1.1935536, 1.3225864,...","[0.6654362, 0.6609094, 0.87366796, 1.0230516, ...","[0.2327341, 0.3381231, 0.5568053, 0.74650556, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.27034205, 0.4041477, 0.45876226, 0.52429974...","[0.8194214, 0.80269855, 0.71908414, 0.64104396...","[0.8116741, 0.7202179, 0.6230456, 0.64590967, ...","[0.7982271, 1.0675378, 1.1112098, 0.8346204, 1...","[0.882244, 1.2294496, 1.485585, 1.2322956, 1.0..."
2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,...,"[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[1126742, 1326741], [3526740, 3726740], [5826...","[[192745, 249350], [389747, 407142], [917544, ...","[[33037711, 33038706], [33264908, 33270313], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-0.058899105, 0.19379705, 0.72198904, 1.09058...","[0.052017204, 0.2566182, 0.409202, 0.2288757, ...","[0.0039443844, 0.2524406, 0.48910367, 0.408243...","[0.0017428675, 0.040085953, 0.1289722, 0.19345...","[0.24099746, 0.36528546, 0.31526712, -0.039408..."
3,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,...,"[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[1126742, 1326741], [3526740, 3726740], [5826...","[[192745, 249350], [389747, 407142], [917544, ...","[[33037711, 33038706], [33264908, 33270313], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-0.5686467, -0.88410044, -0.7741067, -0.08716...","[-0.7133093, -1.0918, -0.88557106, -0.07278667...","[-0.38970518, -0.8093877, -0.46165076, 0.39570...","[0.22564748, -0.09320222, 0.1

# Power Calcuation

- Getting the column name of all the traces

In [15]:
trace_columns = [col for col in LFP_TRACES_DF.columns if "lfp_trace" in col]

In [16]:
trace_columns

['mPFC_lfp_trace',
 'MD_lfp_trace',
 'LH_lfp_trace',
 'BLA_lfp_trace',
 'vHPC_lfp_trace']

In [17]:
for col in trace_columns:
    print(col)
    LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(np.float16))

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


- Calcuating the power at each frequency band

In [18]:
for col in trace_columns:
    brain_region = col.split("_")[0]
    print(brain_region)

    # Define column names
    lfp_trace_col = f"{brain_region}_lfp_trace"
    multitaper_col = f"{brain_region}_power_multitaper"
    connectivity_col = f"{brain_region}_power_connectivity"
    frequencies_col = f"{brain_region}_power_frequencies"
    power_col = f"{brain_region}_power_all_frequencies_all_windows"
    
    try:
        # Apply Multitaper function to the lfp_trace column
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF[lfp_trace_col].apply(
            lambda x: Multitaper(
                time_series=x, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT,
                time_window_duration=TIME_WINDOW_DURATION, 
                time_window_step=TIME_WINDOW_STEP
            )
        )

        # Apply Connectivity function to the multitaper column
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and power functions to the connectivity column
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.power().squeeze()
        )
        
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[power_col].apply(lambda x: x.astype(np.float16))
            
        # Removing unnecessary columns
        LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")
    
    except Exception as e: 
        print(e)

mPFC
MD
LH
BLA
vHPC


- Getting the timestamps of the power

In [19]:
LFP_TRACES_DF["power_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])
# .iloc[0][500:-500:500].shape

- Making sure that the timestamps for power makes sense with shape and values

In [20]:
LFP_TRACES_DF["power_timestamps"].head().apply(lambda x: x.shape)

0    (6828,)
1    (6828,)
2    (6833,)
3    (6833,)
4    (6840,)
Name: power_timestamps, dtype: object

In [21]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_all_frequencies_all_windows" in col][0]].iloc[0].shape

(6828, 500)

In [22]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

array([       0,       20,       40, ..., 68293300, 68293320, 68293340],
      dtype=int32)

In [23]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_timestamps" in col][0]].iloc[0]

array([   10000,    20000,    30000, ..., 68260000, 68270000, 68280000],
      dtype=int32)

- Checking if the right frequencies are being used

In [24]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col]].head()

,mPFC_power_frequencies,MD_power_frequencies,LH_power_frequencies,BLA_power_frequencies,vHPC_power_frequencies
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [25]:
LFP_TRACES_DF["power_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [26]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col], errors="ignore")

In [27]:
LFP_TRACES_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,...,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,mPFC_power_all_frequencies_all_windows,MD_power_all_frequencies_all_windows,LH_power_all_frequencies_all_windows,BLA_power_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,power_timestamps,power_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,...,"[0.8945, 0.962, 1.193, 1.322, 1.038, 0.572, 0....","[0.6655, 0.661, 0.8735, 1.023, 0.7085, 0.2559,...","[0.2328, 0.3381, 0.5566, 0.7466, 0.8315, 0.822...","[[0.041679501256392006, 0.08025313321269073, 0...","[[0.00632052951397403, 0.017458768312512106, 0...","[[0.019591267649446943, 0.04858695953405047, 0...","[[0.03760241292264982, 0.024253130490369527, 0...","[[0.009636655939867502, 0.005739269661452429, ...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,...,"[0.8115, 0.72, 0.623, 0.646, 0.652, 0.3887, 0....","[0.7983, 1.067, 1.111, 0.8345, 1.065, 1.497, 1...","[0.8823, 1.2295, 1.485, 1.232, 1.067, 1.286, 1...","[[0.00025556702544395277, 0.000589562549611339...","[[0.012225516178285791, 0.012737223370812326, ...","[[0.016422261794743077, 0.017655437752233688, ...","[[0.00047484018178471524, 0.002134975374615609...","[[0.0012576934645483337, 0.0018907147707882734...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,...,"[0.003944, 0.2524, 0.489, 0.4082, 0.215, -0.04...","[0.001743, 0.0401, 0.129, 0.1935, 0.1063, -0.1...","[0.241, 0.3652, 0.3152, -0.0394, -0.2773, -0.2...","[[0.021773998806363992, 0.009880744836821182, ...","[[0.009157147551752981, 0.0061473868756681755,...","[[0.01685006691527802, 0.01132764040782633, 0....","[[0.008236900067186664, 0.005802347564020453, ...","[[0.0010641609787729585, 0.006975338107397395,...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,...,"[-0.3896, -0.8096, -0.4617, 0.3958, 0.04495, -...","[0.2257, -0.0932, 0.1667, 0.7603, 0.5005, 0.14...","[1.249, 1.02, 0.978, 1.081, 0.8228, 0.563, 0.4...","[[0.18142789322995392, 0.2056980254527757, 0.2...","[[0.23281909011984986, 0.23291718437601186, 0....","[[0.36859342646059795, 0.36857278882253264, 0....","[[0.3391271592403716, 0.2897

## Coherence Calculation

- Getting the trace column pairs

In [28]:
trace_columns

['mPFC_lfp_trace',
 'MD_lfp_trace',
 'LH_lfp_trace',
 'BLA_lfp_trace',
 'vHPC_lfp_trace']

In [29]:
brain_region_pairs = generate_pairs(sorted(trace_columns))
brain_region_pairs = sorted(brain_region_pairs)


In [30]:
brain_region_pairs

[('BLA_lfp_trace', 'LH_lfp_trace'),
 ('BLA_lfp_trace', 'MD_lfp_trace'),
 ('BLA_lfp_trace', 'mPFC_lfp_trace'),
 ('BLA_lfp_trace', 'vHPC_lfp_trace'),
 ('LH_lfp_trace', 'MD_lfp_trace'),
 ('LH_lfp_trace', 'mPFC_lfp_trace'),
 ('LH_lfp_trace', 'vHPC_lfp_trace'),
 ('MD_lfp_trace', 'mPFC_lfp_trace'),
 ('MD_lfp_trace', 'vHPC_lfp_trace'),
 ('mPFC_lfp_trace', 'vHPC_lfp_trace')]

## Coherece Calculation

- Calculating the coherence

In [31]:
for region_1, region_2 in brain_region_pairs:
    # Define base name for pair
    pair_base_name = f"{region_1.split('_')[0]}_{region_2.split('_')[0]}"
    print(pair_base_name)

    try:
        # Define column names
        multitaper_col = f"{pair_base_name}_coherence_multitaper"
        connectivity_col = f"{pair_base_name}_coherence_connectivity"
        frequencies_col = f"{pair_base_name}_coherence_frequencies"
        coherence_col = f"{pair_base_name}_coherence_all_frequencies_all_windows"

        # Apply Multitaper function
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF.apply(
            lambda x: Multitaper(
                time_series=np.array([x[region_1], x[region_2]]).T, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                time_window_step=TIME_WINDOW_STEP, 
                time_window_duration=TIME_WINDOW_DURATION
            ), 
            axis=1
        )

        # Apply Connectivity function
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and coherence functions
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        LFP_TRACES_DF[coherence_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.coherence_magnitude()[:,:,0,1]
        )

        LFP_TRACES_DF[coherence_col] = LFP_TRACES_DF[coherence_col].apply(lambda x: x.astype(np.float16))

    except Exception as e: 
        print(e)

    # Drop temporary columns
    LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")

BLA_LH
BLA_MD
BLA_mPFC
BLA_vHPC
LH_MD
LH_mPFC
LH_vHPC
MD_mPFC
MD_vHPC
mPFC_vHPC


- Getting the timestamps of the coherence

In [32]:
LFP_TRACES_DF["coherence_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for coherence makes sense with shape and values

In [33]:
LFP_TRACES_DF["coherence_timestamps"].head().apply(lambda x: x.shape)

0    (6828,)
1    (6828,)
2    (6833,)
3    (6833,)
4    (6840,)
Name: coherence_timestamps, dtype: object

In [34]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_all_frequencies_all_windows" in col][0]].iloc[0].shape

(6828, 500)

In [35]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

array([       0,       20,       40, ..., 68293300, 68293320, 68293340],
      dtype=int32)

In [36]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_timestamps" in col][0]].iloc[0]

array([   10000,    20000,    30000, ..., 68260000, 68270000, 68280000],
      dtype=int32)

- Checking if the right frequencies are being used

In [37]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col]].head()

,BLA_LH_coherence_frequencies,BLA_MD_coherence_frequencies,BLA_mPFC_coherence_frequencies,BLA_vHPC_coherence_frequencies,LH_MD_coherence_frequencies,LH_mPFC_coherence_frequencies,LH_vHPC_coherence_frequencies,MD_mPFC_coherence_frequencies,MD_vHPC_coherence_frequencies,mPFC_vHPC_coherence_frequencies
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [38]:
LFP_TRACES_DF["coherence_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [39]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col], errors="ignore")

In [40]:
LFP_TRACES_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,...,BLA_mPFC_coherence_all_frequencies_all_windows,BLA_vHPC_coherence_all_frequencies_all_windows,LH_MD_coherence_all_frequencies_all_windows,LH_mPFC_coherence_all_frequencies_all_windows,LH_vHPC_coherence_all_frequencies_all_windows,MD_mPFC_coherence_all_frequencies_all_windows,MD_vHPC_coherence_all_frequencies_all_windows,mPFC_vHPC_coherence_all_frequencies_all_windows,coherence_timestamps,coherence_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,...,"[[0.8882462472542707, 0.7449939553643623, 0.76...","[[0.864650389664005, 0.9488239972287765, 0.859...","[[0.898235409549242, 0.9753945571389674, 0.979...","[[0.9847727734682262, 0.9976162808882771, 0.99...","[[0.5126532369357578, 0.6707339707827665, 0.67...","[[0.959942388246991, 0.9680483018076077, 0.974...","[[0.8122942136743958, 0.6315233933466671, 0.60...","[[0.6344141863898974, 0.713746795228113, 0.682...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,...,"[[0.37218350872674577, 0.5474877937004712, 0.8...","[[0.9931020941333979, 0.938495298486667, 0.967...","[[0.9971447666729005, 0.9976798571743588, 0.99...","[[0.010947253062853622, 0.26438278548278926, 0...","[[0.08765185817511825, 0.8494980522658399, 0.1...","[[0.01811359528451841, 0.24024780431789414, 0....","[[0.05985489135991847, 0.8143333066679522, 0.1...","[[0.29440426411996323, 0.40080798525577577, 0....","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,...,"[[0.982312128809607, 0.8764566286696479, 0.626...","[[0.24421425316496992, 0.35825412590379235, 0....","[[0.9985306340754755, 0.9926722435280122, 0.94...","[[0.9909263641220544, 0.86863316679752, 0.5106...","[[0.38497009777198254, 0.3590567914611752, 0.3...","[[0.996376622424479, 0.8596055493088888, 0.671...","[[0.3480330618802691, 0.39228829835192347, 0.3...","[[0.29917225889774096, 0.10483028070910669, 0....","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,...,"[[0.8718237112031685, 0.9593661773824126, 0.94...","[[0.940944846478928, 0.9901225016406486, 0.971...","[[0.9871513986484733, 0.9951282378909503, 0.99...","[[0.9196267939872201, 0.98376270967600

In [41]:
LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_coherence.pkl")

# Calculate Granger's

In [43]:
for region_1, region_2 in brain_region_pairs:
    # Define base name for pair
    region_1_base_name = region_1.split('_')[0]
    region_2_base_name = region_2.split('_')[0]

    pair_base_name = f"{region_1_base_name}_{region_2_base_name}"
    print(pair_base_name)

    try:
        # Define column names
        multitaper_col = f"{pair_base_name}_granger_multitaper"
        connectivity_col = f"{pair_base_name}_granger_connectivity"
        frequencies_col = f"{pair_base_name}_granger_frequencies"
        granger_1_2_col = f"{region_1_base_name}_{region_2_base_name}_granger_all_frequencies_all_windows"
        granger_2_1_col = f"{region_2_base_name}_{region_1_base_name}_granger_all_frequencies_all_windows"

        # Apply Multitaper function
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF.apply(
            lambda x: Multitaper(
                time_series=np.array([x[region_1], x[region_2]]).T, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                time_window_step=TIME_WINDOW_STEP, 
                time_window_duration=TIME_WINDOW_DURATION
            ), 
            axis=1
        )
    
        # Apply Connectivity function
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and granger functions
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        
        LFP_TRACES_DF[granger_1_2_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.pairwise_spectral_granger_prediction()[:,:,0,1]
        )

        LFP_TRACES_DF[granger_2_1_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.pairwise_spectral_granger_prediction()[:,:,1,0]
        )

        LFP_TRACES_DF[granger_1_2_col] = LFP_TRACES_DF[granger_1_2_col].apply(lambda x: x.astype(np.float16))
        LFP_TRACES_DF[granger_2_1_col] = LFP_TRACES_DF[granger_2_1_col].apply(lambda x: x.astype(np.float16))
        
    except Exception as e: 
        print(e)

    # Drop temporary columns
    LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")

BLA_LH


Maximum iterations reached. 6753 of 6828 converged
Maximum iterations reached. 6814 of 6828 converged
Maximum iterations reached. 6812 of 6833 converged
Maximum iterations reached. 6831 of 6833 converged
Maximum iterations reached. 6814 of 6840 converged
Maximum iterations reached. 6837 of 6841 converged
Maximum iterations reached. 6788 of 6804 converged
Maximum iterations reached. 6376 of 6678 converged
Maximum iterations reached. 6725 of 6746 converged
Maximum iterations reached. 6743 of 6746 converged
Maximum iterations reached. 7939 of 7968 converged
Maximum iterations reached. 7959 of 7968 converged
Maximum iterations reached. 6241 of 6559 converged
Maximum iterations reached. 6709 of 6858 converged
Maximum iterations reached. 3898 of 4117 converged
Maximum iterations reached. 6787 of 6835 converged
Maximum iterations reached. 6816 of 6835 converged
Maximum iterations reached. 6978 of 6979 converged
Maximum iterations reached. 6720 of 7034 converged
Maximum iterations reached. 718

BLA_MD


Maximum iterations reached. 6779 of 6828 converged
Maximum iterations reached. 6819 of 6828 converged
Maximum iterations reached. 6810 of 6833 converged
Maximum iterations reached. 6828 of 6833 converged
Maximum iterations reached. 6805 of 6840 converged
Maximum iterations reached. 6836 of 6841 converged
Maximum iterations reached. 6786 of 6804 converged
Maximum iterations reached. 6381 of 6678 converged
Maximum iterations reached. 6734 of 6746 converged
Maximum iterations reached. 6742 of 6746 converged
Maximum iterations reached. 7928 of 7968 converged
Maximum iterations reached. 7947 of 7968 converged
Maximum iterations reached. 6238 of 6559 converged
Maximum iterations reached. 6709 of 6858 converged
Maximum iterations reached. 3898 of 4117 converged
Maximum iterations reached. 6780 of 6835 converged
Maximum iterations reached. 6819 of 6835 converged
Maximum iterations reached. 6976 of 6979 converged
Maximum iterations reached. 6714 of 7034 converged
Maximum iterations reached. 718

BLA_mPFC


Maximum iterations reached. 6686 of 6828 converged
Maximum iterations reached. 6832 of 6833 converged
Maximum iterations reached. 6839 of 6840 converged
Maximum iterations reached. 6802 of 6804 converged
Maximum iterations reached. 6376 of 6678 converged
Maximum iterations reached. 6733 of 6746 converged
Maximum iterations reached. 7967 of 7968 converged
Maximum iterations reached. 7964 of 7968 converged
Maximum iterations reached. 6274 of 6559 converged
Maximum iterations reached. 6709 of 6858 converged
Maximum iterations reached. 3897 of 4117 converged
Maximum iterations reached. 6825 of 6835 converged
Maximum iterations reached. 6724 of 7034 converged
Maximum iterations reached. 7184 of 7204 converged
Maximum iterations reached. 7145 of 7175 converged
Maximum iterations reached. 7172 of 7175 converged
Maximum iterations reached. 7227 of 7241 converged


KeyboardInterrupt: 

- Getting the timestamps of the granger

In [ ]:
LFP_TRACES_DF["granger_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for granger makes sense with shape and values

In [ ]:
LFP_TRACES_DF["granger_timestamps"].head().apply(lambda x: x.shape)

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_all_frequencies_all_windows" in col][0]].iloc[0].shape

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_timestamps" in col][0]].iloc[0]

- Checking if the right frequencies are being used

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col]].head()

In [ ]:
LFP_TRACES_DF["granger_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [ ]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col], errors="ignore")

In [ ]:
LFP_TRACES_DF.head()

In [ ]:
LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_granger.pkl")

In [ ]:
raise ValueError()